In [1]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")

In [2]:
import os
import torch

from pytorch_ie import Document, Pipeline
from src.models.span_clf_with_gazetteer import SpanClassificationWithGazetteerModel
from src.taskmodules.span_clf_with_gazetteer import SpanClassificationWithGazetteerTaskModule
from src.datamodules.datasets.multiconer import load_multiconer

from eval_multiconer import seqeval_score

In [3]:
def average_models(checkpoint1, checkpoint2):
    model1 = SpanClassificationWithGazetteerModel.load_from_checkpoint(checkpoint1)
    model2 = SpanClassificationWithGazetteerModel.load_from_checkpoint(checkpoint2)
    
    beta = 0.5 #The interpolation parameter    
    params1 = model1.named_parameters()
    params2 = model2.named_parameters()

    dict_params2 = dict(params2)

    for name1, param1 in params1:
        # if name1 in dict_params2:
        dict_params2[name1].data.copy_(beta*param1.data + (1-beta)*dict_params2[name1].data)

    model1.load_state_dict(dict_params2, strict=False)
    
    return model1
    
    
#     assert len(model_checkpoints) >= 1
    
#     models = [SpanClassificationWithGazetteerModel.load_from_checkpoint(checkpoint) for checkpoint in model_checkpoints]
    
#     num_models = len(models)
    
#     model1 = models[0]
    
#     with torch.inference_mode():
#         for name1, param1 in model1.named_parameters():
#             for i in range(1, num_models):
#                 if name1 in models[i].named_parameters():
#                     param_i = models[i].named_parameters()[name1]
#                     param1.data.add_(param_i.data)

#         for name1, param1 in model1.named_parameters():
#             param1.div_(num_models)
    
#     return model1

In [4]:
MODEL_PATH = "/media/christoph/HDD/models/multiconer-en-spanclf-wiki-gazetteer-freeze_3/"
DATASET_DIR = "/home/christoph/Downloads/public_data/"

DATASET_SPLIT = "validation"
FILTER_ENTITIES = False

CHECKPOINT = "multiconer-en-15-val_f1-0.91.ckpt"

CUDA_DEVICE = 0

In [5]:
taskmodule = SpanClassificationWithGazetteerTaskModule.from_pretrained(MODEL_PATH, use_efficient_gazetteer=False)

In [10]:
model = average_models(
    # "/media/christoph/HDD/models/multiconer-en-spanclf-wiki-gazetteer-freeze_1/multiconer-en-06-val_f1-0.89.ckpt",
    "/media/christoph/HDD/models/multiconer-en-spanclf-wiki-gazetteer-freeze_2/multiconer-en-08-val_f1-0.89.ckpt",
    "/media/christoph/HDD/models/multiconer-en-spanclf-wiki-gazetteer-freeze_3/multiconer-en-09-val_f1-0.89.ckpt"
)

pipeline = Pipeline(model=model, taskmodule=taskmodule, device=CUDA_DEVICE)

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discri

In [11]:
eval_documents = load_multiconer(
    data_dir=DATASET_DIR,
    name="en",
    split=DATASET_SPLIT,
)

predict_field = "entities"

pipeline(eval_documents, predict_field=predict_field)

print(seqeval_score(documents=eval_documents, predict_field=predict_field, filter_entities=FILTER_ENTITIES))

Using custom data configuration en-b2b539f3793511b0
Reusing dataset multi_co_ner (/home/christoph/.cache/huggingface/datasets/multi_co_ner/en-b2b539f3793511b0/1.0.0/afa61df806aafde79b4bd38aef1a3db19216190e1aa77a223a2d70d1eea327c9)
100%|██████████| 800/800 [00:00<00:00, 15988.96it/s]
800it [00:00, 939.03it/s]
/home/christoph/miniconda3/envs/multi_coner/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2242: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/christoph/miniconda3/envs/multi_coner/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2242: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/home/christoph/miniconda3/envs/multi_coner/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2242: UserWarnin

              precision    recall  f1-score   support

        CORP     0.9392    0.8808    0.9091       193
          CW     0.8547    0.8352    0.8448       176
         GRP     0.9072    0.9263    0.9167       190
         LOC     0.8966    0.8889    0.8927       234
         PER     0.9721    0.9621    0.9671       290
        PROD     0.7911    0.8503    0.8197       147

   micro avg     0.9028    0.8984    0.9006      1230
   macro avg     0.8935    0.8906    0.8917      1230
weighted avg     0.9041    0.8984    0.9009      1230



In [12]:
model.save_pretrained("/media/christoph/HDD/models/multiconer-en-spanclf-wiki-gazetteer-freeze-avg/")

In [6]:
model2 = SpanClassificationWithGazetteerModel.load_from_checkpoint(os.path.join(MODEL_PATH, CHECKPOINT))
pipeline2 = Pipeline(model=model2, taskmodule=taskmodule, device=CUDA_DEVICE)

eval_documents = load_multiconer(
    data_dir=DATASET_DIR,
    name="en",
    split=DATASET_SPLIT,
)

predict_field = "entities"

pipeline2(eval_documents, predict_field=predict_field)

print(seqeval_score(documents=eval_documents, predict_field=predict_field, filter_entities=FILTER_ENTITIES))

Some weights of the model checkpoint at google/electra-large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration en-b2b539f3793511b0
Reusing dataset multi_co_ner (/home/christoph/.cache/huggingface/datasets/multi_co_ner/en-b2b539f3793511b0/1.0.0/afa61df806aafde79b4bd38aef1a3db19216190e1aa77a223a2d70d1eea327c9)
10

              precision    recall  f1-score   support

        CORP     0.8854    0.8808    0.8831       193
          CW     0.9045    0.8068    0.8529       176
         GRP     0.9185    0.8895    0.9037       190
         LOC     0.8979    0.9017    0.8998       234
         PER     0.9790    0.9655    0.9722       290
        PROD     0.8121    0.8231    0.8176       147

   micro avg     0.9086    0.8886    0.8985      1230
   macro avg     0.8996    0.8779    0.8882      1230
weighted avg     0.9089    0.8886    0.8983      1230

